In [9]:
# importing libraries to work with data 
import pandas as pd
import numpy as np
import datetime
import requests
import json
from tqdm import tqdm

In [2]:
# getting the list of subject IDs having adverse effect
subjects_ae = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/list')
subjects_ae_json = subjects_ae.json()
subjects_ae = pd.DataFrame(subjects_ae_json['data'], columns = ['subject_id'])

In [3]:
# getting AE data for all subject IDs
ae_df = pd.DataFrame()
for subject_id in subjects_ae['subject_id']:
    response_ae = requests.get(f'https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/ae/subject/{subject_id}/list')
    response_ae_json = response_ae.json()
    ae_df = ae_df.append(response_ae_json['data'], ignore_index='true')

In [4]:
# getting the list of subject IDs that have taken concomitant medication
subjects_cm = requests.get('https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/list')
subjects_cm_json= subjects_cm.json()
subjects_cm = pd.DataFrame(subjects_cm_json['data'], columns = ['subject_id'])

In [5]:
# getting CM data for all subject IDs
cm_df = pd.DataFrame()
for subject_id in subjects_cm['subject_id']:
    response_cm = requests.get(f'https://pyhack-dot-pharmanlp-177020.uc.r.appspot.com/api/1/StudyHack/cm/subject/{subject_id}/list')
    response_cm_json = response_cm.json()
    cm_df = cm_df.append(response_cm_json['data'], ignore_index='true')

In [6]:
# converting the date columns into a readable format to clean
ae_df['aestdat'] = pd.to_datetime(ae_df['aestdat'], errors = 'coerce', format = '%d-%b-%y')
ae_df['aeendat'] = pd.to_datetime(ae_df['aeendat'], errors = 'coerce', format = '%d-%b-%y')

cm_df['cmstdat'] = pd.to_datetime(cm_df['cmstdat'], errors = 'coerce', format = '%d-%b-%y')
cm_df['cmendat'] = pd.to_datetime(cm_df['cmendat'], errors = 'coerce', format = '%d-%b-%y')

In [7]:
# removing unwanted rows and columns from AE data
ae = ae_df[['siteid', 'subjectid', 'formid', 'aeterm', 'aespid', 'aestdat', 'aeendat', 'aecmgiv']]
ae = ae.dropna(subset=['aestdat', 'aeendat'])

index1 = ae[ae['aecmgiv'] == 'NO'].index
ae.drop(index1, inplace = True)

In [8]:
# removing unwanted rows and columns from CM data
cm = cm_df[['siteid', 'subjectid', 'formid', 'cmtrt', 'cmaeno', 'cmstdat', 'cmendat', 'cmaer']]
cm = cm.dropna(subset=['cmstdat', 'cmendat']) 

index2 = cm[cm['cmaer'] == 'NO'].index
cm.drop(index2, inplace = True)

In [10]:
# function to remove 0 from list
def remove_items(test_list, item=0):
    res = [i for i in test_list if i != item]
    return res

dummy = pd.DataFrame(cm.cmaeno)
dummy = dummy.dropna(axis=0)

cm_int = pd.DataFrame()

# convert CMAENO values to float or list and store in a separate dataframe
for i in tqdm(dummy.cmaeno):
    i = pd.to_numeric(i, errors='ignore')
    if i == 'nan':
        continue
    if type(i) == str:
        i = i.split(',')
    cm_int = cm_int.append(pd.Series(i), ignore_index=True)
    
# replace nan values to 0    
cm_int.replace({np.nan:0}, inplace=True)

# new column to store CMAENO values after conversion
cm_int['combined'] = cm_int.values.tolist()

# removing 0 from the list
dummy1 = pd.DataFrame(cm_int['combined'].apply(remove_items))

100%|█████████████████████████████████████████████████████████████████████████████| 7212/7212 [00:20<00:00, 355.07it/s]


In [11]:
# replace CMAENO column in the original dataframe with the values after conversion
cm_replace = pd.DataFrame(cm)
cm_replace['cmaeno'] = dummy1.combined.values

In [12]:
# remove empty lists in CMAENO column
cm_replace = cm_replace[cm_replace['cmaeno'].map(len) > 0]

In [13]:
# convert lists in the CMAENO column to float
cm_clean = pd.DataFrame(columns=['siteid', 'subjectid', 'formid', 'cmtrt', 'cmaer', 'cmstdat', 'cmendat','cmaeno'])

for i,j in tqdm(zip(cm_replace['cmaeno'], cm_replace.index)):
    if len(i) > 1:
        for k in i:
            cm_clean = cm_clean.append({'siteid':cm_replace.siteid[j],
                                             'subjectid':cm_replace.subjectid[j],
                                             'formid':cm_replace.formid[j],
                                             'cmtrt':cm_replace.cmtrt[j],
                                             'cmaer':cm_replace.cmaer[j],
                                             'cmstdat':cm_replace.cmstdat[j],
                                             'cmendat':cm_replace.cmendat[j],
                                             'cmaeno':k}, ignore_index=True)
    else:
        cm_clean = cm_clean.append({'siteid':cm_replace.siteid[j],
                                             'subjectid':cm_replace.subjectid[j],
                                             'formid':cm_replace.formid[j],
                                             'cmtrt':cm_replace.cmtrt[j],
                                             'cmaer':cm_replace.cmaer[j],
                                             'cmstdat':cm_replace.cmstdat[j],
                                             'cmendat':cm_replace.cmendat[j],
                                             'cmaeno':cm_replace.cmaeno[j][0]},ignore_index=True)

6761it [00:46, 146.67it/s]


In [14]:
# removing non-float values
for i, j in zip(cm_clean.cmaeno, cm_clean.index):
    if type(i) == str:
        cm_clean = cm_clean.drop(index=j, axis=0)

In [15]:
# convert type of column for merging
cm_clean.cmaeno = pd.to_numeric(cm_clean.cmaeno)
cm = cm_clean

In [16]:
# merging AE and CM dataframes
final = pd.merge(ae, cm, how = 'inner', left_on = ['subjectid', 'aespid'], right_on = ['subjectid', 'cmaeno'])
final

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno
0,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,CASPOFUNGIN ACETATE,YES,2018-05-04,2018-05-21,6.0
1,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,VANCOMYCIN HYDROCHLORIDE,YES,2018-05-05,2018-05-21,6.0
2,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,LENOGRASTIM,YES,2018-05-17,2018-05-20,6.0
3,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,POLYETHYLENE GLYCOL TREATED HUMAN NORMAL IMMUN...,YES,2018-05-17,2018-05-19,6.0
4,5173,76809,38768,DRUG ERUPTION,9.0,2018-05-27,2018-06-06,YES,5173,38623,OLOPATADINE HYDROCHLORIDE,YES,2018-05-27,2018-05-29,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3017,5303,18042999,38768,HYPOKALEMIA,4.0,2020-01-28,2020-01-30,YES,5303,38623,KALIUM 2mmol/h,YES,2020-01-30,2020-01-30,4.0
3018,5303,18042999,38768,SEPSIS,22.0,2020-02-10,2020-02-10,YES,5303,38623,AMBISOME 250mg,YES,2020-01-29,2020-02-09,22.0
3019,5018,18282353,38768,FEBRILE NEUTROPENIA,17.0,2020-03-08,2020-03-11,YES,5018,38623,LEUCOSTIM,YES,2020-03-08,2020-03-09,17.0
3020,5018,18282353,38768,CATHETER-ASSOCIATED SOFT TISSUE INFECTION,4.0,2020-01-23,2020-01-29,YES,5018,38623,AMIKACIN,YES,2020-01-23,2020-02-05,4.0


In [17]:
# type 1 discrepancy result
final['discrepancy'] = final['aestdat'] > final['cmstdat']
type1 = final.drop(final[final['discrepancy'] == False].index)
type1

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno,discrepancy
39,5086,138700,38768,FEBRILE NEUTROPENIA,3.0,2018-06-20,2018-06-25,YES,5086,38623,PIPERACILLIN,YES,2018-06-13,2018-06-20,3.0,True
56,5178,166621,38768,TYPHLITIS,5.0,2018-07-09,2018-09-18,YES,5178,38623,METRONIDAZOLE,YES,2018-07-06,2018-07-08,5.0,True
63,5178,166621,38768,SEPSIS,13.0,2018-08-21,2018-08-28,YES,5178,38623,FILGRASTIM-SNDZ,YES,2018-08-20,2018-08-24,13.0,True
77,5535,231407,38768,FEVER,7.0,2018-07-19,2018-07-20,YES,5535,38623,ACETAMINOPHEN,YES,2018-06-19,2018-07-19,7.0,True
112,5086,386442,38768,FEBRILE NEUTROPENIA,1.0,2018-08-20,2018-09-03,YES,5086,38623,CEFEPIME,YES,2018-08-17,2018-09-03,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2945,5233,17418518,38768,NEUTROPENIC FEVER,1.0,2020-03-09,2020-03-15,YES,5233,38623,SODIUM CHLORIDE IV BOLUS,YES,2020-01-24,2020-01-24,1.0,True
2946,5233,17418518,38768,NEUTROPENIC FEVER,1.0,2020-03-09,2020-03-15,YES,5233,38623,TOBRAMYCIN,YES,2020-01-25,2020-01-25,1.0,True
2961,5114,17739603,38768,ORAL MUCOSITIS,2.0,2020-01-10,2020-03-14,YES,5114,38623,HYPROMORPHONE,YES,2000-01-22,2020-01-23,2.0,True
3001,5045,17956076,38768,LEUKOPENIA,5.0,2020-02-01,2020-02-13,YES,5045,38623,ACCOFIL,YES,2020-01-31,2020-02-14,5.0,True


In [18]:
# type 2 discrepancy result
final['discrepancy'] = final['cmstdat'] > final['aeendat']
type2 = final.drop(final[final['discrepancy'] == False].index)
type2

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno,discrepancy
62,5178,166621,38768,HYPOPHOSPHATEMIA,20.0,2018-07-06,2018-07-06,YES,5178,38623,PHOSPHORUS 250-298-45 MG TABLET,YES,2018-07-21,2018-07-23,20.0,True
94,5537,265932,38768,FEBRILE NEUTROPENIA,24.0,2018-08-22,2018-09-03,YES,5537,38623,MEROPENEM HYDRATE,YES,2018-09-06,2018-09-06,24.0,True
104,5178,305637,35537,ATRIAL FLUTTER/TACHYCARDIA RAPID VENTRICULAR RATE,1.0,2018-09-03,2018-09-04,YES,5178,38623,DILTIAZEM HCL,YES,2018-09-05,2018-09-05,1.0,True
266,5522,954328,35537,VOMITING,8.0,2019-02-04,2019-02-12,YES,5522,38623,METOCLOPRAMIDE,YES,2019-05-29,2019-06-19,8.0,True
267,5522,954328,35537,VOMITING,8.0,2019-02-04,2019-02-12,YES,5522,38623,METOCLOPRAMIDE,YES,2019-02-19,2019-02-22,8.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2940,5521,17263384,38768,BONE PAIN,6.0,2020-01-17,2020-01-17,YES,5521,38623,Oxycodone,YES,2020-01-18,2020-01-18,6.0,True
2949,5233,17418518,38768,CONSTIPATION,3.0,2020-01-11,2020-01-12,YES,5233,38623,MIRALAX,YES,2020-01-13,2020-02-02,3.0,True
2986,5045,17954820,38768,FEVER,4.0,2020-02-12,2020-02-12,YES,5045,38623,MEROPENEM,YES,2020-02-13,2020-02-17,4.0,True
2987,5045,17954820,38768,FEVER,4.0,2020-02-12,2020-02-12,YES,5045,38623,SUMAMED,YES,2020-02-18,2020-02-20,4.0,True


In [19]:
# type 3 discrepancy result
type3 = final[final.duplicated(['aestdat', 'aeterm'])]
type3

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno,discrepancy
1,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,VANCOMYCIN HYDROCHLORIDE,YES,2018-05-05,2018-05-21,6.0,False
2,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,LENOGRASTIM,YES,2018-05-17,2018-05-20,6.0,False
3,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,POLYETHYLENE GLYCOL TREATED HUMAN NORMAL IMMUN...,YES,2018-05-17,2018-05-19,6.0,False
5,5173,76809,38768,DRUG ERUPTION,9.0,2018-05-27,2018-06-06,YES,5173,38623,EBASTINE,YES,2018-05-29,2018-06-04,9.0,False
6,5173,76809,38768,DRUG ERUPTION,9.0,2018-05-27,2018-06-06,YES,5173,38623,DIPHENHYDRAMINE,YES,2018-05-27,2018-06-06,9.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3004,5045,17956076,38768,FEVER,6.0,2020-02-09,2020-02-09,YES,5045,38623,ALGOPYRIN (METAMIZOL SODIUM),YES,2020-02-09,2020-02-09,6.0,False
3005,5045,17956076,38768,HYPOKALEMIA,7.0,2020-01-30,2020-02-06,YES,5045,38623,POTASSIUM,YES,2020-02-03,2020-02-03,7.0,False
3009,5045,17956076,38768,FEVER,19.0,2020-03-22,2020-03-22,YES,5045,38623,ALGOPYRIN (METAMIZOLE SODIUM),YES,2020-03-22,2020-03-22,19.0,False
3016,5045,17956076,38768,HYPOKALEMIA,28.0,2020-06-02,2020-06-05,YES,5045,38623,KALDYUM,YES,2020-06-05,2020-06-07,28.0,False


In [20]:
# type 4 discrepancy result
type4 = final[final.duplicated(['cmtrt', 'cmstdat'])]
type4

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno,discrepancy
51,5178,166621,38768,INTERMITTENT PAROXYSMAL ATRIAL FIBRILLATION WI...,1.0,2018-07-04,2018-07-17,YES,5178,38623,DILTIAZEM,YES,2018-07-05,2018-07-05,1.0,False
52,5178,166621,38768,INTERMITTENT PAROXYSMAL ATRIAL FIBRILLATION WI...,1.0,2018-07-04,2018-07-17,YES,5178,38623,DILTIAZEM,YES,2018-07-05,2018-07-05,1.0,False
164,5086,475581,38768,FEBRILE NEUTROPENIA,2.0,2018-09-15,2018-09-18,YES,5086,38623,VANCOMYCIN,YES,2018-09-18,2018-09-18,2.0,False
165,5086,475581,38768,FEBRILE NEUTROPENIA,2.0,2018-09-15,2018-09-18,YES,5086,38623,VANCOMYCIN,YES,2018-09-18,2018-09-18,2.0,False
176,5086,475581,38768,ABDOMINAL PAIN,13.0,2018-09-25,2018-09-26,YES,5086,38623,HYDROMORPHONE,YES,2018-09-25,2018-09-26,13.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2645,5218,9975024,38768,INTRACRANIAL HEMORRHAGE,1.0,2019-08-19,2019-08-19,YES,5218,38623,ADRENALINE HYDROCHLORIDE INJECTION,YES,2019-08-19,2019-08-19,1.0,False
2866,5134,16357906,38768,FEVER,27.0,2020-03-21,2020-03-31,YES,5134,38623,Ceftazidime Pentahydrate,YES,2020-03-25,2020-03-31,27.0,False
2931,5252,17165682,38768,HYPOTENSION,31.0,2020-01-04,2020-01-13,YES,5252,38623,VANCOMYCIN,YES,2020-01-06,2020-01-13,31.0,False
2972,5114,17739603,38768,HEADACHE,3.0,2020-01-13,2020-02-18,YES,5114,38623,COMPAZINE,YES,2020-02-06,2020-02-06,3.0,False


In [21]:
# type 5 discrepancy result
final['discrepancy'] = final['aeendat'] > final['cmendat']
type5 = final.drop(final[final['discrepancy'] == False].index)
type5

,siteid_x,subjectid,formid_x,aeterm,aespid,aestdat,aeendat,aecmgiv,siteid_y,formid_y,cmtrt,cmaer,cmstdat,cmendat,cmaeno,discrepancy
2,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,LENOGRASTIM,YES,2018-05-17,2018-05-20,6.0,True
3,5173,76809,38768,FEBRILE NEUTROPENIA,6.0,2018-05-04,2018-05-21,YES,5173,38623,POLYETHYLENE GLYCOL TREATED HUMAN NORMAL IMMUN...,YES,2018-05-17,2018-05-19,6.0,True
4,5173,76809,38768,DRUG ERUPTION,9.0,2018-05-27,2018-06-06,YES,5173,38623,OLOPATADINE HYDROCHLORIDE,YES,2018-05-27,2018-05-29,9.0,True
5,5173,76809,38768,DRUG ERUPTION,9.0,2018-05-27,2018-06-06,YES,5173,38623,EBASTINE,YES,2018-05-29,2018-06-04,9.0,True
8,5173,76809,38768,ABDOMINALDISCOMFORT,18.0,2018-05-31,2018-06-08,YES,5173,38623,DAIKENCHUTO,YES,2018-05-31,2018-06-01,18.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2997,5045,17954820,38768,HYPOKALEMIA,25.0,2020-05-22,2020-06-02,YES,5045,38623,POTASSIUM,YES,2020-05-29,2020-05-29,25.0,True
3005,5045,17956076,38768,HYPOKALEMIA,7.0,2020-01-30,2020-02-06,YES,5045,38623,POTASSIUM,YES,2020-02-03,2020-02-03,7.0,True
3015,5045,17956076,38768,HYPOKALEMIA,28.0,2020-06-02,2020-06-05,YES,5045,38623,POTASSIUM,YES,2020-06-02,2020-06-02,28.0,True
3018,5303,18042999,38768,SEPSIS,22.0,2020-02-10,2020-02-10,YES,5303,38623,AMBISOME 250mg,YES,2020-01-29,2020-02-09,22.0,True
